# Tutorial 1: Your First Training Run

This tutorial walks you through the `examples/01_run_hello_world.py` script. You will learn the fundamental workflow of ToolBrain: defining tools, creating an agent, setting up the `Brain`, and running a training loop.

## The Goal

We will train a small language model (`Qwen/Qwen2.5-0.5B-Instruct`) to correctly use two simple Python functions: `add` and `multiply`. We'll use Reinforcement Learning (RL) to teach the agent to provide the correct answer for a given query.

## The Code: `01_run_hello_world.py`

Let's break down the script step by step.

### Step 1: Define Tools

First, we define the functions that our agent can use. The `@tool` decorator from `smolagents` is used to mark them as tools.

In [2]:
from smolagents import tool, TransformersModel, CodeAgent
from toolbrain import Brain
from toolbrain.rewards import reward_exact_match

/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/toolbrain/adapters.py:9: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
#from smolagents import tool
@tool
def add(a: int, b: int) -> int:
    """
    Add two integers.

    Args:
        a (int): First addend.
        b (int): Second addend.

    Returns:
        int: Sum of a and b.
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): First factor.
        b (int): Second factor.

    Returns:
        int: Product of a and b.
    """
    return a * b


Clear *docstrings* and *type hints* are crucial. The agent uses this information to understand what each tool does and how to use it.

### Step 2: Prepare Training Data

Next, we create a simple dataset. Each item contains a `query` for the agent and a `gold_answer` that we can use to verify the agent's performance.


In [5]:
training_dataset = [
    {
        "query": "Use the add tool to calculate 5 + 7",
        "gold_answer": "12"
    },
    {
        "query": "What is 8 multiplied by 6?",
        "gold_answer": "48"
    },
]

### Step 3: Create the Agent

Now, we create the agent that will learn to use our tools. This involves two parts:

1.  **`TransformersModel`**: A `smolagents` model wrapper that uses the Hugging Face *Transformers* library to run an LLM locally (you typically pass a `model_id`; it forwards generation kwargs like `temperature`, etc.).  [Hugging Face](https://huggingface.co/docs/smolagents/en/reference/models#smolagents.TransformersModel)  
2.  **`CodeAgent`**: The agent class itself; you initialize it with a `model` and a list of `tools`. It writes tool calls as **Python code** and executes them (locally or in a sandbox).  [Hugging Face](https://huggingface.co/docs/smolagents/en/guided_tour)


In [8]:
# Load the language model
model = TransformersModel(
    model_id="Qwen/Qwen2.5-0.5B-Instruct",
    max_new_tokens=128
)

# Create the agent
agent = CodeAgent(
    model=model,
    tools=[add, multiply],
    max_steps=1  # The maximum number of ReAct loop iterations (each is think → tool call → observe) the agent may take during a single run(...). 
)

### Step 4: Create the Brain

The `Brain` is the core of ToolBrain. It orchestrates the training process. We initialize it with our agent and define the training strategy.

In [9]:
from toolbrain import Brain
from toolbrain.rewards import reward_exact_match

brain = Brain(
    agent,                          # The agent to be trained
    algorithm="GRPO",               # The learning algorithm to use
    reward_func=reward_exact_match  # The function to score agent performance
)

- **`agent`**: The `CodeAgent` instance we just created.
- **`algorithm="GRPO"`**: We choose GRPO as the reinforcement learning algorithm to train the agent for this task.
- **`reward_func=reward_exact_match`**: We use a built-in reward function. It will give a reward of `1.0` if the agent's final answer is exactly the same as the `gold_answer` in our dataset, and `0.0` otherwise.

### Step 5: Run the Training

Finally, we call the `train()` method on our `brain` instance. This will start the training loop.


In [11]:
brain.train(training_dataset, num_iterations=10)



🚀 Starting training...

--- Iteration 1/10 ---

🔄 Training step for query: 'Use the add tool to calculate 5 + 7...'
Calling get_trace with query: 'Use the add tool to calculate 5 + 7...'
Enable tool retrieval: False
    📝 Trace 1/2


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the add tool to calculate 5 + 7                                                                             │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-0.5B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = add(5, 7)                                                                                               
  final_answer(str(result))                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  product = 8 * 6                                                                                                  
  final_answer(product)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 48

[Step 1: Duration 0.90 seconds| Input tokens: 2,019 | Output tokens: 52]

  🎯 Computing rewards for 2 traces...
      Using single-trace reward function
      🎯 Trace 1 Reward: 1.000
      🎯 Trace 2 Reward: 1.000
📈 Sliding window avg reward (last 10): 0.9000
  🧠 Running RL training step with 2 traces...
  ✅ RL training step completed

🎉 Training finished!


You will see output logs showing the agent's progress. For each training step, it will:
1.  Generate one or more **traces** by attempting the query.
2.  Compute a **reward** for each trace.
3.  Run the **GRPO** algorithm to update the model.

You'll see a `Sliding window avg reward` metric, which should increase as the agent learns the correct behavior.


---

Congratulations! You've just completed your first training run with ToolBrain. In the next tutorials, we'll dive deeper into the concepts of Traces and Reward Functions.